In [ ]:
!pip uninstall apache-beam -y && pip install -U pandas polars pyarrow narwhals 

In [ ]:
import pandas as pd
import polars as pl

pd.options.mode.copy_on_write = True
pd.options.future.infer_string = True

In [ ]:
from typing import Any
import narwhals as nw

def q19(
    lineitem_ds_raw: Any,
    part_ds_raw: Any
    
) -> Any:

    lineitem_ds = nw.from_native(lineitem_ds_raw)
    part_ds = nw.from_native(part_ds_raw)

    result = (
        part_ds.join(lineitem_ds, left_on="p_partkey", right_on="l_partkey")
        .filter(nw.col("l_shipmode").is_in(["AIR", "AIR REG"]))
        .filter(nw.col("l_shipinstruct") == "DELIVER IN PERSON")
        .filter(
            (
                (nw.col("p_brand") == "Brand#12")
                & nw.col("p_container").is_in(
                    ["SM CASE", "SM BOX", "SM PACK", "SM PKG"]
                )
                & (nw.col("l_quantity").is_between(1, 11))
                & (nw.col("p_size").is_between(1, 5))
            )
            | (
                (nw.col("p_brand") == "Brand#23")
                & nw.col("p_container").is_in(
                    ["MED BAG", "MED BOX", "MED PKG", "MED PACK"]
                )
                & (nw.col("l_quantity").is_between(10, 20))
                & (nw.col("p_size").is_between(1, 10))
            )
            | (
                (nw.col("p_brand") == "Brand#34")
                & nw.col("p_container").is_in(
                    ["LG CASE", "LG BOX", "LG PACK", "LG PKG"]
                )
                & (nw.col("l_quantity").is_between(20, 30))
                & (nw.col("p_size").is_between(1, 15))
            )
        )
        .select(
            (nw.col("l_extendedprice") * (1 - nw.col("l_discount")))
            .sum()
            .round(2)
            .alias("revenue")
        )
    )


    return nw.to_native(result)

In [ ]:
dir_ = "/kaggle/input/tpc-h-data-parquet-s-2/"
lineitem = dir_ + 'lineitem.parquet'
part = dir_ + 'part.parquet'

In [ ]:
IO_FUNCS = {
    'pandas': lambda x: pd.read_parquet(x, engine='pyarrow'),
    'pandas[pyarrow]': lambda x: pd.read_parquet(x, engine='pyarrow', dtype_backend='pyarrow'),
    'polars[eager]': lambda x: pl.read_parquet(x),
    'polars[lazy]': lambda x: pl.scan_parquet(x),
}

In [ ]:
results = {}

## pandas via Narwhals

In [ ]:
tool = 'pandas'
fn = IO_FUNCS[tool]
timings = %timeit -o -q q19(fn(lineitem), fn(part))
results[tool] = timings.all_runs

## pandas, pyarrow dtypes, via Narwhals

In [ ]:
tool = 'pandas[pyarrow]'
fn = IO_FUNCS[tool]
timings = %timeit -o -q q19(fn(lineitem), fn(part))
results[tool] = timings.all_runs

## Polars read_parquet

In [ ]:
tool = 'polars[eager]'
fn = IO_FUNCS[tool]
timings = %timeit -o -q q19(fn(lineitem), fn(part))
results[tool] = timings.all_runs

## Polars scan_parquet

In [ ]:
tool = 'polars[lazy]'
fn = IO_FUNCS[tool]
timings = %timeit -o -q q19(fn(lineitem), fn(part)).collect()
results[tool] = timings.all_runs

## Save

In [ ]:
import json
with open('results.json', 'w') as fd:
    json.dump(results, fd)
